In [114]:
import pandas as pd
from datetime import datetime
import numpy as np

In [115]:
df = pd.read_csv('../../csv/AMZN.csv', index_col=0)

In [116]:
# 결측치, 양의 무한대, 음의 무한대 제거
flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
df = df.loc[~flag, ]

In [117]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [118]:
'Date' in df.columns

False

In [119]:
# 인덱스가 Date가 아니라면 Date 컬럼을 인덱스로 변경
# 조건식: 컬럼들 중 Date가 존재한다면
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

- 인덱스를 시계열 데이터로 변경
    1. pandas의 to_datetime() 함수 이용
    2. datetime의 strptime() 함수 & map 함수 이용

In [120]:
# index를 시계열데이터로 변경
# pandas에 내장된 to_datetime() 함수 이용
pd.to_datetime(df.index)

DatetimeIndex(['1997-05-15', '1997-05-16', '1997-05-19', '1997-05-20',
               '1997-05-21', '1997-05-22', '1997-05-23', '1997-05-27',
               '1997-05-28', '1997-05-29',
               ...
               '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14',
               '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20',
               '2019-06-21', '2019-06-24'],
              dtype='datetime64[ns]', name='Date', length=5563, freq=None)

In [121]:
# datetime 라이브러리 안에 있는 strptime() 함수 이용
datetime.strptime(df.index[0], '%Y-%m-%d')

datetime.datetime(1997, 5, 15, 0, 0)

In [122]:
# strptime() 함수와 map 함수 사용
df.index = df.index.map(
    lambda x : datetime.strptime(x, '%Y-%m-%d')
)

In [123]:
# 투자의 시작 시간과 종료 시간을 통해서 df의 인덱스를 필터링
start = '2010-01-01'
end = '2015-01-01'
t_s = datetime.strptime(start, '%Y-%m-%d')
t_e = datetime.strptime(end, '%Y-%m-%d')

In [124]:
df.loc[t_s : t_e, ]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,136.250000,136.610001,133.139999,133.899994,133.899994,7599900
2010-01-05,133.429993,135.479996,131.809998,134.690002,134.690002,8851900
2010-01-06,134.600006,134.729996,131.649994,132.250000,132.250000,7178800
2010-01-07,132.009995,132.320007,128.800003,130.000000,130.000000,11030200
2010-01-08,130.559998,133.679993,129.029999,133.520004,133.520004,9830500
...,...,...,...,...,...,...
2014-12-24,306.380005,307.000000,302.880005,303.029999,303.029999,1513800
2014-12-26,305.000000,310.779999,303.809998,309.089996,309.089996,2893800
2014-12-29,307.850006,314.269989,306.579987,312.040009,312.040009,3009000


In [125]:
price_df = df.loc[start : end, ['Adj Close']]

In [126]:
# 첫날(구매)의 수정종가, 마지막날(판매)의 수정종가
price_df.iloc[-1, 0]  / price_df.iloc[0, 0]     # 마지막날 / 첫날

np.float64(2.317774607219176)

In [127]:
# pct_change(): 일별 수익률(전일대비 수익률)
price_df['rtn'] = (price_df['Adj Close'].pct_change() + 1).fillna(1)

In [128]:
# 최종 누적 수익률
price_df['rtn'].cumprod()[-1]

C:\Users\toqha\AppData\Local\Temp\ipykernel_17012\121289533.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_df['rtn'].cumprod()[-1]


np.float64(2.317774607219178)

#### Buy and Hold 함수화
- 매개변수 개수는 4개
    - _df : 데이터프레임 -> 필수
    - _start : 투자 시작 시간(매수) -> '2010-01-01' 기본값 설정
    - _end : 투자 종료 시간(매도) -> 현재 시간 datetime의 now() 기본값 설정
    - _col : 특정 컬럼 선택 -> 'Adj Close'(수정종가) 기본값 설정
1. _df의 복사본 생성 (df 변수 생성) -> 원본 데이터 유지 위해
2. 컬럼 중 'Date'가 존재한다면 Date 컬럼을 인덱스로 변경
3. 인덱스를 시계열 데이터로 변경
4. 결측치와 양의 무한대, 음의 무한대 데이터 제거
5. _start와 _end를 기준으로 인덱스 필터링, _col을 기준으로 컬럼 필터링
    - 시도하고 문제가 발생한다면 '인자값이 잘못되었다' 출력하고 함수 종료
6. 일별 수익률(rtn)을 생성하여 pct_change() + 1 데이터 대입
7. 누적 수익률을 계산하여 새로운 컬럼 (acc_rtn)에 대입
8. 만들어진 데이터프레임과 최종 누적수익률 반환

In [129]:
datetime.now()

datetime.datetime(2025, 6, 17, 11, 25, 45, 903776)

In [130]:
def bnh(_df, _start='2010-01-01', _end=datetime.now(), _col='Adj Close'):
    df = _df.copy()
    # try:
    #     _start = datetime.strptime(_start, '%Y-%m-%d')
    #     # 만약 _end 타입이 문자라면
    #     if type(_end) == 'str':
    #         _end = datetime.strptime(_start, '%y-%m-%d')
    # except:
    #     print('시간 포맷이 맞지 않습니다, (YYYY-mm-dd)')
    #     return
    
    # Date가 컬럼에 존재한다면 Date를 인덱스로 변경
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)

    # 인덱스 -> 시계열데이터로 변경
    df.index = pd.to_datetime(df.index)
    
    # 결측치, 양의 무한대, 음의 무한대 제거
    flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
    df = df.loc[~flag, ]
    
    try: 
       df = df.loc[_start : _end, [_col]]
    except Exception as e:
        print(e)
        print('인자값이 잘못되었습니다')
        return ""
    
    # 일별 수익률 컬럼 생성
    df['rtn'] = (df[_col].pct_change() + 1).fillna(1)
    # 누적 수익률 컬럼 생성
    df['acc_rtn'] = df['rtn'].cumprod()
    # 최종 누적 수익률
    final_acc_rtn = df.iloc[-1, -1]

    # 결과 데이터프레임과 최종 누적수익률 반환
    return df, final_acc_rtn

In [131]:
test_df = pd.read_csv('../../csv/MSFT.csv')

In [132]:
bnh(test_df, _start='2015-01-01', _col='Close')

(                 Close       rtn   acc_rtn
 Date                                      
 2015-01-02   46.759998  1.000000  1.000000
 2015-01-05   46.330002  0.990804  0.990804
 2015-01-06   45.650002  0.985323  0.976262
 2015-01-07   46.230000  1.012705  0.988666
 2015-01-08   47.590000  1.029418  1.017750
 ...                ...       ...       ...
 2019-06-18  135.160004  1.017388  2.890505
 2019-06-19  135.690002  1.003921  2.901839
 2019-06-20  136.949997  1.009286  2.928785
 2019-06-21  136.970001  1.000146  2.929213
 2019-06-24  138.289993  1.009637  2.957442
 
 [1126 rows x 3 columns],
 np.float64(2.9574422351344074))

In [133]:
msft, far = bnh(test_df, _start='2015-01-01', _col='Close')

In [134]:
msft

,Close,rtn,acc_rtn
Date,,,
2015-01-02,46.759998,1.000000,1.000000
2015-01-05,46.330002,0.990804,0.990804
2015-01-06,45.650002,0.985323,0.976262
2015-01-07,46.230000,1.012705,0.988666
2015-01-08,47.590000,1.029418,1.017750
...,...,...,...
2019-06-18,135.160004,1.017388,2.890505
2019-06-19,135.690002,1.003921,2.901839
2019-06-20,136.949997,1.009286,2.928785


In [135]:
far

np.float64(2.9574422351344074)